In [3]:
import errno
import pickle
import importlib.util
import random
import re
import time
from datetime import datetime
from typing import Union, Tuple

import imageio
import numpy as np
from fairmotion.ops import conversions
from torch import nn

from bullet_agent import SimAgent
from real_time_runner import RTRunner
from real_time_runner_minimal import RTRunnerMin

import torch
import os
import argparse

# make deterministic
from data_utils import \
    viz_current_frame_and_store_fk_info_include_fixed, \
    loss_angle, loss_j_pos, loss_root_dist_pos, loss_max_jerk, loss_root_jerk, our_pose_2_bullet_format
from render_funcs import init_viz, COLOR_OURS, update_height_field_pb, set_color, COLOR_GT
from learning_utils import set_seed
import constants as cst

In [4]:
torch.set_num_threads(1)
np.set_printoptions(threshold=10_000, precision=10)
torch.set_printoptions(threshold=10_000, precision=10)

In [12]:
parser = argparse.ArgumentParser(description='Run our model and related works models')
parser.add_argument('--ours_path_name_kin', type=str, default="model-kin-amass-4-knee-v2.pt",
                    help='')
parser.add_argument('--name_contains', type=str, default='',
                    help='Please use "" to be able to pass multiple search keys split by whitespaces')
parser.add_argument('--test_len', type=int, default=600,
                    help='')
parser.add_argument('--render', action='store_true',
                    help='')
parser.add_argument('--compare_gt', action='store_true',
                    help='')
parser.add_argument('--seed', type=int, default=42,
                    help='')
parser.add_argument('--five_sbp', action='store_true',
                    help='')
parser.add_argument('--with_acc_sum', action='store_true',
                    help='')
parser.add_argument('--viz_terrain', action='store_true',
                    help='')
parser.add_argument('--data_version_tag', type=str, default=None,
                    help='')
# parser.add_argument('--save_c', action='store_true',
#                     help='')                # for the DIP-IMU set which has C info
# args = parser.parse_args()

MODEL_NAME="model-without-dip9and10"
MODEL_NAME_WGT=f"output/{MODEL_NAME}.pt"

args = parser.parse_args([
    "--name_contains", "dipimu_s_09 dipimu_s_10",
    "--ours_path_name_kin", MODEL_NAME_WGT,
    "--with_acc_sum",  
    "--test_len", "30000",
    "--compare_gt", 
    "--seed", "42",
    "--five_sbp",
    "--data_version_tag", "v1"
                         ])

In [14]:
set_seed(args.seed)

TEST_LEN = args.test_len
RENDER = args.render
MAX_TEST_MOTION_PRE_CAT = 50        # make testing faster
# if args.save_c:
#     MAX_TEST_MOTION_PRE_CAT = 50000
# else:
#     MAX_TEST_MOTION_PRE_CAT = 50
USE_5_SBP = args.five_sbp
WITH_ACC_SUM = args.with_acc_sum

MAP_BOUND = cst.MAP_BOUND * 2.0     # some motions are in large range
GRID_NUM = int(MAP_BOUND/cst.GRID_SIZE) * 2

In [15]:
def run_ours_wrapper_with_c_rt(imu, s_gt, model_name, char) -> (np.ndarray, np.ndarray):
    def load_model(name):
        from simple_transformer_with_state import TF_RNN_Past_State
        input_channels_imu = 6 * (9 + 3)
        if USE_5_SBP:
            output_channels = 18 * 6 + 3 + 20
        else:
            output_channels = 18 * 6 + 3 + 8

        model = TF_RNN_Past_State(
            input_channels_imu, output_channels,
            rnn_hid_size=512,
            tf_hid_size=1024, tf_in_dim=256,
            n_heads=16, tf_layers=4,
            dropout=0.0, in_dropout=0.0,
            past_state_dropout=0.8,
            with_acc_sum=WITH_ACC_SUM
        )
        model.load_state_dict(torch.load(name))
        model = model.cuda()
        # model.eval()
        return model

    m = load_model(model_name)

    # ours_out, c_out, viz_locs_out = test_run_ours_gpt_v4_with_c_rt(char, s_gt, imu, m, 40)
    ours_out, c_out, viz_locs_out = test_run_ours_gpt_v4_with_c_rt_minimal(char, s_gt, imu, m, 40)

    return ours_out, c_out, viz_locs_out


def test_run_ours_gpt_v4_with_c_rt_minimal(
        char: SimAgent,
        s_gt: np.array,
        imu: np.array,
        m: nn.Module,
        max_win_len: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:

    # use real time runner with offline data
    rt_runner = RTRunnerMin(
        char, m, max_win_len, s_gt[0],
        with_acc_sum=WITH_ACC_SUM,
    )

    m_len = imu.shape[0]
    s_traj_pred = np.zeros((m_len, cst.n_dofs * 2))
    s_traj_pred[0] = s_gt[0]

    c_traj_pred = np.zeros((m_len, rt_runner.n_sbps * 4))
    viz_locs_seq = [np.ones((rt_runner.n_sbps, 3)) * 100.0]

    for t in range(0, m_len-1):
        res = rt_runner.step(imu[t, :], s_traj_pred[t, :3])

        s_traj_pred[t + 1, :] = res['qdq']
        c_traj_pred[t + 1, :] = res['ct']

        viz_locs = res['viz_locs']
        for sbp_i in range(viz_locs.shape[0]):
            viz_point(viz_locs[sbp_i, :], sbp_i)
        viz_locs_seq.append(viz_locs)

        if RENDER:
            time.sleep(1. / 180)

    # throw away first "trim" predictions (our algorithm gives dummy values)... append dummy value in the end.
    viz_locs_seq = np.array(viz_locs_seq)
    assert len(viz_locs_seq) == len(s_traj_pred)

    # +2 because post-processing moving average filter effectively introduce a bit more delay
    trim = rt_runner.IMU_n_smooth + 2
    s_traj_pred[0:-trim, :] = s_traj_pred[trim:, :]
    s_traj_pred[-trim:, :] = s_traj_pred[-trim-1, :]
    viz_locs_seq[0:-trim, :, :] = viz_locs_seq[trim:, :, :]
    viz_locs_seq[-trim:, :, :] = viz_locs_seq[-trim-1, :, :]

    return s_traj_pred, c_traj_pred, viz_locs_seq


def test_run_ours_gpt_v4_with_c_rt(
        char: SimAgent,
        s_gt: np.array,
        imu: np.array,
        m: nn.Module,
        max_win_len: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:

    global h_id, h_b_id

    # use real time runner with offline data
    rt_runner = RTRunner(
        char, m, max_win_len, s_gt[0],
        map_bound=MAP_BOUND,
        grid_size=cst.GRID_SIZE,
        play_back_gt=False,
        five_sbp=USE_5_SBP,
        with_acc_sum=WITH_ACC_SUM,
        multi_sbp_terrain_and_correction=False
    )

    m_len = imu.shape[0]
    s_traj_pred = np.zeros((m_len, cst.n_dofs * 2))
    c_traj_pred = np.zeros((m_len, rt_runner.n_sbps * 4))
    s_traj_pred[0] = s_gt[0]

    viz_locs_seq = [np.ones((rt_runner.n_sbps, 3)) * 100.0]

    for t in range(0, m_len-1):
        res = rt_runner.step(imu[t, :], s_traj_pred[t, :3], t=t)

        s_traj_pred[t + 1, :] = res['qdq']
        c_traj_pred[t + 1, :] = res['ct']

        viz_locs = res['viz_locs']
        for sbp_i in range(viz_locs.shape[0]):
            viz_point(viz_locs[sbp_i, :], sbp_i)

        viz_locs_seq.append(viz_locs)

        if t % 15 == 0 and h_id is not None:
            # TODO: double for loop...
            for ii in range(init_grid_np.shape[0]):
                for jj in range(init_grid_np.shape[1]):
                    init_grid_list[jj*init_grid_np.shape[0]+ii] = \
                        rt_runner.region_height_list[rt_runner.height_region_map[ii, jj]]
            h_id, h_b_id = update_height_field_pb(
                pb_client,
                h_data=init_grid_list,
                scale=cst.GRID_SIZE,
                terrainShape=h_id,
                terrain=h_b_id
            )

        if RENDER:
            time.sleep(1. / 180)

    # throw away first "trim" predictions (our algorithm gives dummy values)... append dummy value in the end.
    viz_locs_seq = np.array(viz_locs_seq)

    # +2 because post-processing moving average filter effectively introduce a bit more delay
    trim = rt_runner.IMU_n_smooth + 2
    s_traj_pred[0:-trim, :] = s_traj_pred[trim:, :]
    s_traj_pred[-trim:, :] = s_traj_pred[-trim-1, :]
    viz_locs_seq[0:-trim, :, :] = viz_locs_seq[trim:, :, :]
    viz_locs_seq[-trim:, :, :] = viz_locs_seq[-trim-1, :, :]

    return s_traj_pred, c_traj_pred, viz_locs_seq


def viz_2_trajs_and_return_fk_records_with_sbp(
        char1: SimAgent,
        char2: SimAgent,
        traj1: np.ndarray,
        traj2: np.ndarray,
        start_t: int,
        end_t: int,
        gui: bool,
        seq_c_viz: Union[np.ndarray, None],
) -> (np.ndarray, np.ndarray, np.ndarray, np.ndarray):

    m_len = len(traj1)      # use first length if mismatch

    pq_g_1_s = []
    pq_g_2_s = []

    for t in range(start_t, m_len-end_t):

        pq_g_2 = viz_current_frame_and_store_fk_info_include_fixed(char2, traj2[t])
        pq_g_1 = viz_current_frame_and_store_fk_info_include_fixed(char1, traj1[t])   # GT in grey

        pq_g_1_s.append(pq_g_1)
        pq_g_2_s.append(pq_g_2)

        if seq_c_viz is not None:
            cur_c_viz = seq_c_viz[t, :, :]
            for sbp_i in range(cur_c_viz.shape[0]):
                viz_point(cur_c_viz[sbp_i, :], sbp_i)

        if gui:
            time.sleep(1. / 180)

    return traj1[start_t: m_len-end_t], traj2[start_t: m_len-end_t], np.array(pq_g_1_s), np.array(pq_g_2_s)


def post_processing_our_model(
        char: SimAgent,
        ours_out: np.ndarray) -> np.ndarray:
    poses_post = []
    for pose in ours_out:
        pose_post = our_pose_2_bullet_format(char, pose)
        poses_post.append(pose_post.tolist())
    poses_post = np.array(poses_post)

    return poses_post


def viz_point(x, ind):
    pb_client.resetBasePositionAndOrientation(
        VIDs[ind],
        x,
        [0., 0, 0, 1]
    )


def get_all_testing_filenames(name_contains_list):
    file_paths = []
    for src_dir in imu_readings_dirs_OUR_format:
        src_dir = os.path.join("data", src_dir)
        # list_dirs = [x[0] for x in os.walk(src_dir)]
        # for d in list_dirs:
        with os.scandir(src_dir) as it:
            for entry in it:
                n = entry.name
                if n.endswith('pkl'):
                    f_path = os.path.join(src_dir, n)
                    for name_contains in name_contains_list:
                        if re.search(name_contains, f_path, re.IGNORECASE):
                            file_paths.append(f_path)

                            break
                        # break to here
    return file_paths

In [16]:
"""
    main
"""

imu_readings_dirs_OUR_format = [
    "syn_AMASS_CMU_v0", "syn_Eyes_Japan_Dataset_v0",
    "syn_KIT_v0", "syn_HUMAN4D_v0",
    "syn_ACCAD_v0", "syn_DFaust_67_v0", "syn_HumanEva_v0", "syn_MPI_Limits_v0",
    "syn_MPI_mosh_v0", "syn_SFU_v0", "syn_Transitions_mocap_v0",
    "preprocessed_DIP_IMU_v0", "preprocessed_TotalCapture_v0", "syn_TotalCapture_v0",
    "syn_DanceDB_v0"
]

In [29]:
if args.data_version_tag is not None:
    imu_readings_dirs_OUR_format = [ name.replace("v0", args.data_version_tag) for name in imu_readings_dirs_OUR_format ]

# if args.save_c:
#     try:
#         os.makedirs("../release/data/preprocessed_DIP_IMU_v0_c")    # store c here
#     except FileExistsError:
#         print("warning: path existed")
#     except OSError:
#         exit()

''' Load Character Info Moudle '''
spec = importlib.util.spec_from_file_location(
    "char_info", "amass_char_info.py")
char_info = importlib.util.module_from_spec(spec)
spec.loader.exec_module(char_info)

name_contains_l = args.name_contains.split()
print(name_contains_l)
test_files = get_all_testing_filenames(name_contains_l)
print(len(test_files))
if len(test_files) > MAX_TEST_MOTION_PRE_CAT:
    test_files = random.sample(test_files, MAX_TEST_MOTION_PRE_CAT)
print(test_files)

color = COLOR_OURS

['dipimu_s_09', 'dipimu_s_10']
19
['data/preprocessed_DIP_IMU_v1/dipimu_s_09_02_a.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_02.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_03_a.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_09_03_a.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_09_01_c.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_03_b.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_05.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_09_01_b.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_04_b.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_01_c.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_09_03_b.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_01_a.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_04_a.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_04_c.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_09_02_b.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_10_01_b.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_09_01_a.pkl', 'data/preprocessed_DIP_IMU_v1/dipimu_s_09_04.pkl', 'data/preprocesse

In [32]:
# SMPL : 23 joints (22 joint rotations + 1 root xyz)
# -> 20 joints (exclude toes, because this simulation charactor model doesn't have toes)
len(char_info.joint_idx), char_info.joint_idx.keys()

(20,
 odict_keys(['root', 'lhip', 'lknee', 'lankle', 'rhip', 'rknee', 'rankle', 'lowerback', 'upperback', 'chest', 'lowerneck', 'upperneck', 'lclavicle', 'lshoulder', 'lelbow', 'lwrist', 'rclavicle', 'rshoulder', 'relbow', 'rwrist']))

In [19]:
# TODO: really odd, need to be huge for pybullet to work (say. 10.0)
init_grid_np = np.random.uniform(-10.0, 10.0, (GRID_NUM, GRID_NUM))
init_grid_list = list(init_grid_np.flatten())

pb_client, c1, c2, VIDs, h_id, h_b_id = init_viz(char_info,
                                                 init_grid_list,
                                                 hmap_scale=cst.GRID_SIZE,
                                                 gui=RENDER,
                                                 compare_gt=args.compare_gt,
                                                 color=color,
                                                 viz_h_map=args.viz_terrain)

[SimAgent] Creating an agent... data/amass.urdf
[SimAgent] Creating an agent... data/amass.urdf


In [20]:
gt_list = []
ours_list = []
ours_c_list = []
ours_c_viz_list = []
tp_list = []
dip_list = []
test_files_included = []
for f in test_files:

    if not (os.path.exists(f)):
        print("ignored ", f)
        continue

    data = pickle.load(open(f, "rb"))
    X = data['imu']
    Y = data['nimble_qdq']

    # exclude too short trajs
    if Y.shape[0] < 2.5 / cst.DT:
        continue

    # to make all motion equal in stat compute, and run faster
    if Y.shape[0] > TEST_LEN:
        rand_start = random.randrange(0, Y.shape[0] - TEST_LEN)
        start = rand_start
        end = rand_start + TEST_LEN
    else:
        start = 0
        end = Y.shape[0]
    X = X[start: end, :]
    Y = Y[start: end, :]

    # for clearer visualization, amass data not calibrated well wrt floor
    # translation errors are computed from displacement not absolute Y
    Y[:, 2] += 0.05       # move motion root 5 cm up

    # print(X.shape)
    # print(Y.shape)
    # print(start)
    # print(end)
    gt_list.append(Y)
    test_files_included.append(f)
    print(f)

    ours, C, ours_c_viz = run_ours_wrapper_with_c_rt(X, Y, args.ours_path_name_kin, c1)
    ours_list.append(ours)
    ours_c_viz_list.append(ours_c_viz)

    # if args.save_c:
    #     save_name = f.replace("v0", "v0_c")
    #     assert "dipimu" in f
    #     with open(save_name, "wb") as handle:
    #         assert len(X) == len(C)
    #         print(C.shape)
    #         pickle.dump(
    #             {"constrs": C},
    #             handle,
    #             protocol=pickle.HIGHEST_PROTOCOL
    #         )
    #     print("saved", save_name)
    
    break

data/preprocessed_DIP_IMU_v1/dipimu_s_09_02_a.pkl
model with acc sum
number of parameters: %e 3677315


In [23]:
# s_traj_pred, c_traj_pred, viz_locs_seq = test_run_ours_gpt_v4_with_c_rt_minimal(...)


# X : imu R^{12*6} = {(6:rotation + 3:acc_raw + 3:acc_filtered) * 6:IMUs}


# Y : (xyz, v_xyz) R^{57*2}={(20-1):joints *3 *2:(xyz, v_xyz)}
#    "qdq" = {q, dq} <- get_raw_motion_info_nimble_q_dummy_dq(...) : https://github.com/jyf588/transformer-inertial-poser/blob/adef2489c927413228ed4137c8fc58e74b58fa11/data_utils.py#L103

# https://github.com/jyf588/transformer-inertial-poser/blob/adef2489c927413228ed4137c8fc58e74b58fa11/data_utils.py#L152
#         cur_info = p.tolist() + conversions.Q2A(Q).tolist() + j_q_filtered + \  # (1) + (1) + (1): position (xyz, rotation_xyz, omega:rotation_velocity but not used in this method)
#                    v.tolist() + w.tolist() + j_dq_filtered                      # (1) + (1) + (1): velocity (xyz, rotation_xyz, omega:rotation_velocity but not used in this method)

# https://github.com/jyf588/transformer-inertial-poser/blob/adef2489c927413228ed4137c8fc58e74b58fa11/data_utils.py#L159
#         raw_info.shape[1] == ((n_j - 2) * 3 + 3 + 3) * 2                        # bvh : excluded root? or bones?


X.shape, Y.shape

((2766, 72), (2766, 114))

In [48]:
n_j = len(char_info.joint_idx) - 1

t = 50

p,   rot, jq = Y[t][  0:     n_j], Y[t][  n_j: 2*n_j], Y[t][2*n_j: 3*n_j] 
v, v_rot, jq = Y[t][3*n_j: 4*n_j], Y[t][4*n_j: 5*n_j], Y[t][5*n_j: 6*n_j]

In [55]:
n_j, n_j*3*2, Y.shape

(19, 114, (2766, 114))

In [62]:
p,   rot, jq, \
v, v_rot, jq

(array([ 0.0000000000e+00,  0.0000000000e+00,  1.0000000000e+00,
         1.2026493846e+00,  1.1896968088e+00,  1.2280366796e+00,
         2.2138730302e-02, -8.0699120000e-02, -6.2238155678e-02,
         1.1058685927e-03,  1.2425061766e-01,  3.6405972939e-03,
        -1.7372373470e-02,  3.9497703230e-03, -9.4995158459e-03,
        -3.0942233726e-02,  2.8092124635e-02,  4.3664312617e-03,
         4.5304625291e-04]),
 array([-6.0254008695e-02,  3.7774364722e-04,  4.5413941933e-02,
        -1.0353254971e-02, -8.7695608120e-04,  5.9420230920e-02,
        -2.6895332766e-03, -3.5781063244e-01, -1.4022067236e-01,
        -4.0529519310e-02, -1.1163524556e+00,  1.1251337152e-01,
        -1.3060419515e-01, -1.2833978138e-01,  2.6065982226e-02,
         1.3510166233e-02, -2.7034303884e-02,  2.5670629608e-02,
        -1.5341262707e-02]),
 array([ 0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -0.,  0.,  0.,  0.,  0.]),
 array([ 0.          ,  0.          ,  0.        

In [22]:
# s_traj_pred, c_traj_pred, viz_locs_seq = test_run_ours_gpt_v4_with_c_rt_minimal(...)

# s_traj_pred = Y : root_xyz R^{57*2}={(20-1)joints *3 *2}, "qdq" = {q, dq}
# c_traj_pred : SBP_t R^{5*4}, "ct" = {b_t, r_t}
# viz_locs_seq : xyz(SBP_t) R^{5*3}, "viz_locs" = {xyz_t}

ours.shape, C.shape, ours_c_viz.shape  

((2766, 114), (2766, 20), (2766, 5, 3))